<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/01_prompt_engineering/06_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Chat Format


In this notebook, we will explore how we can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

Up to this point, we have learned how to craft clear, specific, and accurate prompts for the LLM (e.g., GPT-4o-mini). However, with newer models such as GPT-5 available directly through OpenAI's ChatGPT platform, one might wonder why we should still write code like this. The reason is that such code can power our own agentic applications, embedding conversational intelligence into tools or services that users can interact with in a daily basis. These prompt templates become building blocks for useful, personalized functionalities. Let's see a simple example in this notebook with an “OrderBot” that can receive and confirm pizza orders, suggest upsells, and handle related queries such as pickup or delivery options, delivery time, and payment methods.

## Setup

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

dotenv_path = find_dotenv() or '/content/OPENAI_API_KEY.env'
load_dotenv(dotenv_path)

openai_api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai_api_key)

In [ ]:
def get_completion_from_messages(messages, model="gpt-4o-mini", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    # print(str(response.choices[0].message))
    return response.choices[0].message.content

In [ ]:
messages =  [
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
{'role':'user', 'content':'tell me a joke'},
{'role':'assistant', 'content':'Why did the chicken cross the road'},
{'role':'user', 'content':'I don\'t know'}  ]

In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To seek the other side, dear friend! Forsooth, 'tis a jest most simple, yet it tickles the fancy. Wouldst thou care for another?


In [ ]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hi Isa! It's nice to meet you. How can I assist you today?


In [ ]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have access to personal data about users unless it has been shared with me in the course of our conversation. If you tell me your name, I can refer to you by it!


In [ ]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa! How can I assist you further today, Isa?


# OrderBot

We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant.

In [ ]:
!pip install -U panel param bokeh jupyter_bokeh

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.HTML(
        f"<div style='color:white'>{prompt}</div>", width=600
    )))
    panels.append(
        pn.Row('Assistant:', pn.pane.HTML(
        f"<div style='background-color:#F6F6F6;padding:6px;border-radius:4px'>{response}</div>",
        width=600)))

    return pn.Column(*panels)

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

# Input and button
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=400),
)

dashboard

Column(sizing_mode='stretch_width')
    [0] TextInput(placeholder='Enter text here…', sizing_mode='stretch_width')
    [1] Row(sizing_mode='stretch_width')
        [0] Button(name='Chat!', sizing_mode='stretch_width')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=400, loading_indicator=True, sizing_mode='stretch_width')

In [ ]:
messages = context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

Here's a summary of your order in JSON format:

```json
{
  "pizza": {
    "type": "pepperoni",
    "size": "large",
    "price": 12.95
  },
  "toppings": [
    {
      "name": "AI sauce",
      "price": 1.50
    }
  ],
  "drinks": [
    {
      "type": "Coca Cola",
      "size": "medium",
      "price": 2.00
    }
  ],
  "sides": [],
  "total_price": 16.45
}
```

Your total comes to $16.45! Would you like to proceed with the payment?
